## Libraries

In [154]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state
from sklearn.datasets import load_boston
import pandas as pd
from pathlib import Path

In [155]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge

In [156]:
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt import UtilityFunction
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [1]:
%run ..//Library//SVRLSPercent_Library.ipynb 

## Load data

In [366]:
boston = load_boston()

# Initializing the dataframe
data = pd.DataFrame(boston.data)

#Adding the feature names to the dataframe
data.columns = boston.feature_names
data.head()

In [ ]:
#Adding target variable to dataframe
data['PRICE'] = boston.target 

In [ ]:
# Spliting target variable and independent variables
X = data.drop(['PRICE'], axis = 1)
y = data['PRICE']

In [ ]:
# Splitting to training and testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 4)

In [ ]:
# Standarize
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Optimization function

In [160]:
def opt_bas(C, gamma):
    
    # parameters
    hyperparameters = {
        'kernel' : "rbf",
        'C' : C, 
        'gamma' : gamma,
    }
    
    # fit and predict
    model = SVR_PerLs(**hyperparameters).fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    # rescale
#     y_pred = scaler1.inverse_transform(y_pred.reshape(-1, 1)).reshape(-1)
    
    # get score
    mape = np.mean(np.abs((y_test - y_pred)/y_test))*100
    
    return -mape

## Run optimization

In [161]:
class newJSONLogger(JSONLogger):

      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"

In [164]:
# Bounded region of parameter space
pbounds = {'C': (9, 10), 'gamma': (0.9, 1.1)}

# Domain reduction function
# bounds_transformer = SequentialDomainReductionTransformer()

# Bayes optimizer instantiation
optimizer = BayesianOptimization(f=opt_bas, 
                                 pbounds=pbounds, 
                                 random_state=1, verbose=2, 
#                                  bounds_transformer=bounds_transformer
                                )

# keep data
log_path = Path().resolve() / "Logs" / "Boston.json"
logger = newJSONLogger(path = str(log_path))
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [ ]:
optimizer.maximize(init_points=20, n_iter=400)

## utils

In [ ]:
## for proving points

# optimizer.probe(params={"C": 131.6135372548593, "gamma": 0.1926931038470609}, lazy = False)
# optimizer.probe(params={"C": 4, "gamma": 2}, lazy = False)
# optimizer.probe(params={"C": 7.628874476243877, "gamma": 1.1659271563361087}, lazy = False)

In [ ]:
### to load data for future predictions

# from bayes_opt.util import load_logs


# new_optimizer = BayesianOptimization(
#     f=black_box_function,
#     pbounds={"x": (-2, 2), "y": (-2, 2)},
#     verbose=2,
#     random_state=7,
# )
# log_path = Path().resolve() / "Logs" / "test2.json"
# # New optimizer is loaded with previously seen points
# load_logs(new_optimizer, logs=["./logs.json"]);